In [3]:
import pandas as pd

In [4]:
# read stations.txt into DataFrame
stationsDF = pd.read_fwf('ghcnd-stations.txt', header=None, delimiter=' '
                         , widths=[12,9,10,7,3,31,4,4,6]
                         , names=['StationID', 'Latitude', 'Longitude', 'Elevation',
                                 'State', 'Name', 'GSN Flag', 'HCN/CRN Flag', 'WMO ID']
                         )

# set StationID as the index
stationsDF.set_index('StationID', inplace=True)

In [5]:
stationsDF.head()

,Latitude,Longitude,Elevation,State,Name,GSN Flag,HCN/CRN Flag,WMO ID
StationID,,,,,,,,
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0
